In [15]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json 
import requests
from bs4 import BeautifulSoup 

In [16]:
text_result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text #get the entire html of the article as a str
html_parsed_result = BeautifulSoup(text_result, 'html.parser') #transform the text to html

neightbourhoodtable = html_parsed_result.find('table', class_ = 'wikitable')
neightbourhoodrows = neightbourhoodtable.find_all('tr')

# extract the info ('Postcode', 'Borough', 'Neighbourhood') from the table
neightbourhoodfinal = []
for row in neightbourhoodrows:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    neightbourhoodfinal.append(info)
    


In [17]:
neighbourhood_df = pd.DataFrame(neightbourhoodfinal[1:], columns=neightbourhoodfinal[0])
neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [18]:
not_assigned_boroughs = neighbourhood_df.index[neighbourhood_df['Borough'] == 'Not assigned']
not_assigned_neighbourhoods = neighbourhood_df.index[neighbourhood_df['Neighbourhood'] == 'Not assigned']
not_assigned_neighbourhoods_and_borough = not_assigned_boroughs & not_assigned_neighbourhoods


neighbourhood_df.drop(neighbourhood_df.index[not_assigned_boroughs], inplace=True)
neighbourhood_df.reset_index(drop=True, inplace=True)

not_assigned_neighbourhoods = neighbourhood_df.index[neighbourhood_df['Neighbourhood'] == 'Not assigned'] # run this again because the indexes on the dataframe where reset

for idx in not_assigned_neighbourhoods:
    neighbourhood_df['Neighbourhood'][idx] = neighbourhood_df['Borough'][idx]
neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [22]:
print('After cleaning the DataFrame, its new shape is {}'.format(neighbourhood_df.shape),'\n')
print('There are:')
print('  {} Postal codes'.format(neighbourhood_df['Postcode'].unique().shape[0]))
print('  {} Boroughs'.format(neighbourhood_df['Borough'].unique().shape[0]))
print('  {} Neighborhoods'.format(neighbourhood_df['Neighbourhood'].unique().shape[0]))

After cleaning the DataFrame, its new shape is (211, 3) 

There are:
  103 Postal codes
  11 Boroughs
  209 Neighborhoods


The dataframe has 103 Postal codes but it has 212 rows, because each Postal code can present more than one neighborhood (210 in total). Therefore, the dataframe should be group by the Postal code, ending with a dataframe with 103 rows.

In [19]:

group = neighbourhood_df.groupby('Postcode')
grouped_neighbourhoods = group['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighbourhoods)))
grouped_df.columns = ['Postcode', 'Borough', 'Neighbourhood']
grouped_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
print('The DataFrame shape is', grouped_df.shape)


The DataFrame shape is (103, 3)
